# Comparative Flux Diagram for Surface Mechanisms

In [2]:
import os
import sys
import numpy as np
import pydot
import rmgpy.tools.fluxdiagram
import rmgpy.chemkin

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


# Settings

In [3]:
# Options controlling the individual flux diagram renderings:
program = 'dot'  # The program to use to lay out the nodes and edges
max_node_count = 35  # The maximum number of nodes to show in the diagram
max_edge_count = 35  # The maximum number of edges to show in the diagram
concentration_tol = 1e-6  # The lowest fractional concentration to show (values below this will appear as zero)
species_rate_tol = 1e-6  # The lowest fractional species rate to show (values below this will appear as zero)
max_node_pen_width = 7.0  # The thickness of the border around a node at maximum concentration
max_edge_pen_width = 9.0  # The thickness of the edge at maximum species rate
radius = 1  # The graph radius to plot around a central species
central_reaction_count = None  # The maximum number of reactions to draw from each central species (None draws all)
# If radius > 1, then this is the number of reactions from every species

# Options controlling the ODE simulations:
initial_time = 1e-12  # The time at which to initiate the simulation, in seconds
time_step = 10 ** 0.1  # The multiplicative factor to use between consecutive time points
abs_tol = 1e-16  # The absolute tolerance to use in the ODE simluations
rel_tol = 1e-8  # The relative tolerance to use in the ODE simulations

# Options controlling the generated movie:
video_fps = 6  # The number of frames per second in the generated movie
initial_padding = 5  # The number of seconds to display the initial fluxes at the start of the video
final_padding = 5  # The number of seconds to display the final fluxes at the end of the video

# Load Mechanisms

In [19]:
# species_path = None
java = False            # always False
settings = None
chemkin_output = ''     # this will be generated automatically
central_species_list = None
superimpose = False     # this will always be false, delete it
save_states = False
read_states = False     # fine to keep this always false and delete relevant code below
diffusion_limited = True
check_duplicates = True

rmg_input_file = '/home/moon/rmg/PR/surface_flux_diagram/input.py'  # for conditions T = 1000

diagram_base_name = 'surface_comparison'
os.makedirs(diagram_base_name, exist_ok=True)
mech_1_inp = '/home/moon/uncertainty_estimator/cpox_pt/cpox_pt_20241101/chem_annotated-gas.inp'
mech_1_surf = '/home/moon/uncertainty_estimator/cpox_pt/cpox_pt_20241101/chem_annotated-surface.inp'
mech_1_dict = '/home/moon/uncertainty_estimator/cpox_pt/cpox_pt_20241101/species_dictionary.txt'
mech_1_label = 'cpox_pt_20241101'

mech_2_inp = '/home/moon/uncertainty_estimator/cpox_pt/cpox_rh_emily/chem_annotated-gas.inp'
mech_2_surf = '/home/moon/uncertainty_estimator/cpox_pt/cpox_rh_emily/chem-surface.inp'
mech_2_dict = '/home/moon/uncertainty_estimator/cpox_pt/cpox_rh_emily/species_dictionary.txt'
mech_2_label = 'cpox_rh_emily'

species_path = '/home/moon/rmg/PR/surface_flux_diagram/species'

generate_images = False
print('Loading RMG job 1...')
rmg_job1 = rmgpy.tools.fluxdiagram.load_rmg_job(
    rmg_input_file,
    mech_1_inp,
    mech_1_dict,
    generate_images=generate_images,
    check_duplicates=check_duplicates,
    surface_path=mech_1_surf
)

print('Loading RMG job 2...')
rmg_job2 = rmgpy.tools.fluxdiagram.load_rmg_job(
    rmg_input_file,
    mech_2_inp,
    mech_2_dict,
    generate_images=generate_images,
    check_duplicates=check_duplicates,
    surface_path=mech_2_surf
)


Loading RMG job 1...
Loading RMG job 2...


# Simulation

In [20]:
print('Conducting simulation of reaction 1')
times1, concentrations1, reaction_rates1 = rmgpy.tools.fluxdiagram.simulate(
    rmg_job1.reaction_model,
    rmg_job1.reaction_systems[0],
    settings
)

print('Conducting simulation of reaction 2')
times2, concentrations2, reaction_rates2 = rmgpy.tools.fluxdiagram.simulate(
    rmg_job2.reaction_model,
    rmg_job2.reaction_systems[0],
    settings
)


Conducting simulation of reaction 1


Conducting simulation of reaction 2


# Compute/assemble species concentrations and fluxes into convenient form

In [21]:
# Get the RMG species and reactions objects
species_list1 = rmg_job1.reaction_model.core.species[:]
reaction_list1 = rmg_job1.reaction_model.core.reactions[:]
num_species1 = len(species_list1)

species_list2 = rmg_job2.reaction_model.core.species[:]
reaction_list2 = rmg_job2.reaction_model.core.reactions[:]
num_species2 = len(species_list2)

In [22]:
# Compute the rates between each pair of species (build up big matrices)
species_rates1 = np.zeros((len(times1), num_species1, num_species1), float)
for index1, reaction1 in enumerate(reaction_list1):
    rate1 = reaction_rates1[:, index1]
    if not reaction1.pairs: reaction1.generate_pairs()
    for reactant1, product1 in reaction1.pairs:
        reactant_index1 = species_list1.index(reactant1)
        product_index1 = species_list1.index(product1)
        species_rates1[:, reactant_index1, product_index1] += rate1
        species_rates1[:, product_index1, reactant_index1] -= rate1
        
species_rates2 = np.zeros((len(times2), num_species2, num_species2), float)
for index2, reaction2 in enumerate(reaction_list2):
    rate2 = reaction_rates2[:, index2]
    if not reaction2.pairs: reaction2.generate_pairs()
    for reactant2, product2 in reaction2.pairs:
        reactant_index2 = species_list2.index(reactant2)
        product_index2 = species_list2.index(product2)
        species_rates2[:, reactant_index2, product_index2] += rate2
        species_rates2[:, product_index2, reactant_index2] -= rate2

In [23]:
# Determine the maximum concentration for each species and the maximum overall concentration
max_concentrations1 = np.max(np.abs(concentrations1), axis=0)
max_concentration1 = np.max(max_concentrations1)

# Determine the maximum reaction rates
max_reaction_rates1 = np.max(np.abs(reaction_rates1), axis=0)

# Determine the maximum rate for each species-species pair and the maximum overall species-species rate
max_species_rates1 = np.max(np.abs(species_rates1), axis=0)
max_species_rate1 = np.max(max_species_rates1)
species_index1 = max_species_rates1.reshape((num_species1 * num_species1)).argsort()


max_concentrations2 = np.max(np.abs(concentrations2), axis=0)
max_concentration2 = np.max(max_concentrations2)

# Determine the maximum reaction rates
max_reaction_rates2 = np.max(np.abs(reaction_rates2), axis=0)

# Determine the maximum rate for each species-species pair and the maximum overall species-species rate
max_species_rates2 = np.max(np.abs(species_rates2), axis=0)
max_species_rate2 = np.max(max_species_rates2)
species_index2 = max_species_rates2.reshape((num_species2 * num_species2)).argsort()


max_species_rate_total = max(max_species_rate1, max_species_rate2)
max_concentration_total = max(max_concentration1, max_concentration2)

# Determine nodes and edges to include for each model

In [24]:
nodes1 = []
edges1 = []
for i in range(num_species1 * num_species1):
    product_index1, reactant_index1 = divmod(species_index1[-i - 1], num_species1)
    if reactant_index1 > product_index1:
        # Both reactant -> product and product -> reactant are in this list, so only keep one of them
        continue
    if max_species_rates1[reactant_index1, product_index1] == 0:
        break
    if reactant_index1 not in nodes1 and len(nodes1) < max_node_count: nodes1.append(reactant_index1)
    if product_index1 not in nodes1 and len(nodes1) < max_node_count: nodes1.append(product_index1)
    if [reactant_index1, product_index1] not in edges1 and [product_index1, reactant_index1] not in edges1:
        edges1.append([reactant_index1, product_index1])
    if len(nodes1) > max_node_count:
        break
    if len(edges1) >= max_edge_count:
        break
        
        
nodes2 = []
edges2 = []
for i in range(num_species2 * num_species2):
    product_index2, reactant_index2 = divmod(species_index2[-i - 1], num_species2)
    if reactant_index2 > product_index2:
        # Both reactant -> product and product -> reactant are in this list, so only keep one of them
        continue
    if max_species_rates2[reactant_index2, product_index2] == 0:
        break
    if reactant_index2 not in nodes2 and len(nodes2) < max_node_count: nodes2.append(reactant_index2)
    if product_index2 not in nodes2 and len(nodes2) < max_node_count: nodes2.append(product_index2)
    if [reactant_index2, product_index2] not in edges2 and [product_index2, reactant_index2] not in edges2:
        edges2.append([reactant_index2, product_index2])
    if len(nodes2) > max_node_count:
        break
    if len(edges2) >= max_edge_count:
        break
         

# Create mapping of species indices between the two models

In [25]:
species2_to_1 = {}
species2_to_1 = {key: value for key, value in zip([x for x in range(len(species_list1))], [-1] * len(species_list1))}
for i in range(len(species_list2)):
    for j in range(len(species_list1)):
        if species_list2[i].is_isomorphic(species_list1[j]):
            species2_to_1[i] = j
            break
    else:
        species2_to_1[i] = -1

species1_to_2 = {}
species1_to_2 = {key: value for key, value in zip([x for x in range(len(species_list1))], [-1] * len(species_list1))}
for i in range(len(species_list1)):
    for j in range(len(species_list2)):
        if species_list1[i].is_isomorphic(species_list2[j]):
            species1_to_2[i] = j
            break
    else:
        species1_to_2[i] = -1
        

In [26]:
# Function to grab and generate the image for a given species
def get_image_path(species):
    species_index = str(species) + '.png'
    image_path = ''
    if not species_path or not os.path.exists(species_path):  # species_path is defined while loading the mechanism
        raise OSError
    for root, dirs, files in os.walk(species_path):
        for f in files:
            if f == species_index:
                image_path = os.path.join(root, f)
                break
    if not image_path:
        image_path = os.path.join(species_path, species_index)
        species.molecule[0].draw(image_path)
    return image_path

In [35]:
[x for x in np.logspace(-15, 0, 20)]

[1e-15,
 6.158482110660255e-15,
 3.7926901907322537e-14,
 2.335721469090121e-13,
 1.438449888287666e-12,
 8.858667904100833e-12,
 5.4555947811685145e-11,
 3.3598182862837877e-10,
 2.06913808111479e-09,
 1.2742749857031347e-08,
 7.847599703514623e-08,
 4.832930238571752e-07,
 2.976351441631313e-06,
 1.8329807108324375e-05,
 0.00011288378916846884,
 0.000695192796177562,
 0.004281332398719396,
 0.026366508987303555,
 0.16237767391887242,
 1.0]

# Build the Graph

In [38]:
# Create the graph
mech1_color = matplotlib.colors.to_hex((1.0, 0.92, 0.0))
mech2_color = matplotlib.colors.to_hex((0.18627451, 0.48823529, 0.94117647))

# add alpha specified by adding hex digits before RGB values
alpha = 0.9
alpha_hex = hex(int(alpha * 255))[2:]
mech1_color = f'#{mech1_color[1:]}{alpha_hex}'
mech2_color = f'#{mech2_color[1:]}{alpha_hex}'

# Grab the fluxes from the time closest (without going over) to each snapshot time
t1_snapshot = 0
t2_snapshot = 0


snapshot_times = np.logspace(-13, -9, 40)
for snapshot_time in snapshot_times:
    t1_snapshot = snapshot_time
    t2_snapshot = snapshot_time

    t1 = np.abs(times1 - t1_snapshot).argmin()
    if times1[t1] > t1_snapshot and t1 != 0:
        t1 -= 1
    t2 = np.abs(times2 - t2_snapshot).argmin()
    if times2[t2] > t2_snapshot and t2 != 0:
        t2 -= 1

    assert -1 not in nodes1
    assert -1 not in nodes2


    slope = -max_node_pen_width / np.log10(concentration_tol)
    graph = pydot.Dot('flux_diagram', graph_type='digraph', overlap="false")
    graph.set_fontname('sans')
    graph.set_fontsize('10')


    # ----------------------------ADD NODES ------------------------------#
    # For Mechanism 1
    for index1 in nodes1:
        nodewidths = np.zeros(2)  # keep track of species concentrations/nodewidths for both mechanisms
        species1 = species_list1[index1]
        node1 = pydot.Node(name=str(species1))
        concentration1 = concentrations1[t1, index1] / max_concentration_total
        if concentration1 < concentration_tol:
            penwidth = 0.0
        else:
            penwidth = round(slope * np.log10(concentration1) + max_node_pen_width, 3)
            nodewidths[0] = penwidth
        node1.set_penwidth(penwidth)
        node1.set_fillcolor('white')
        node1.set_color(mech1_color)
        image_path1 = get_image_path(species1)
        if os.path.exists(image_path1):
            node1.set_image(image_path1)
            node1.set_label("")

        index2 = species1_to_2[index1]
        if index2 >= 0:
            concentration2 = concentrations2[t2, index2] / max_concentration_total
            if concentration2 < concentration_tol:
                penwidth = 0.0
            else:
                penwidth = round(slope * np.log10(concentration2) + max_node_pen_width, 3)
                nodewidths[1] = penwidth
                if node1.get_penwidth() > 0:
                    node1.set_color('black')
                else:
                    node1.set_color(mech2_color)
                    node1.set_penwidth(penwidth)

        if node1.get_color() == 'black':
            node1.set_penwidth(np.average(nodewidths))
        graph.add_node(node1)


    # For Mechanism 2
    for index2 in nodes2:
        if species2_to_1[index2] in nodes1:
            continue  # already took care of it above

        nodewidths = np.zeros(2)
        species2 = species_list2[index2]
        node2 = pydot.Node(name=str(species2))
        concentration2 = concentrations2[t2, index2] / max_concentration_total
        if concentration2 < concentration_tol:
            penwidth = 0.0
        else:
            penwidth = round(slope * np.log10(concentration2) + max_node_pen_width, 3)
            nodewidths[1] = penwidth
        node2.set_fillcolor('white')
        node2.set_color(mech2_color)
        node2.set_penwidth(penwidth)
        # Try to use an image instead of the label
        image_path2 = get_image_path(species2)
        if os.path.exists(image_path2):
            node2.set_image(image_path2)
            node2.set_label("")

        if node2.get_color() == 'black':
            node2.set_penwidth(np.average(nodewidths))

        graph.add_node(node2)


    # ------------------------------- EDGES ------------------------------#
    # Add an edge for each species-species rate
    slope = -max_edge_pen_width / np.log10(species_rate_tol)

    # Go through edges in Mechanism 1
    for reactant_index1, product_index1 in edges1:
        if reactant_index1 in nodes1 and product_index1 in nodes1:
            reactant1 = species_list1[reactant_index1]
            product1 = species_list1[product_index1]

            edge1 = pydot.Edge(str(reactant1), str(product1), color=mech1_color)
            species_rate1 = species_rates1[t1, reactant_index1, product_index1] / max_species_rate_total
            if species_rate1 < 0:
                edge1.set_dir("back")
                species_rate1 = -species_rate1
            else:
                edge1.set_dir("forward")
            # Set the edge pen width
            if species_rate1 < species_rate_tol:
                penwidth = 0.0
                edge1.set_dir("none")
            else:
                penwidth = round(slope * np.log10(species_rate1) + max_edge_pen_width, 3)
            edge1.set_penwidth(penwidth)

            graph.add_edge(edge1)

            # add mech 2
            if species1_to_2[reactant_index1] >= 0 and species1_to_2[product_index1] >= 0:
                reactant_index2 = species1_to_2[reactant_index1]
                product_index2 = species1_to_2[product_index1]

                edge2 = pydot.Edge(str(reactant1), str(product1), color=mech2_color)
                species_rate2 = species_rates2[t2, reactant_index2, product_index2] / max_species_rate_total
                if species_rate2 < 0:
                    edge2.set_dir("back")
                    species_rate2 = -species_rate2
                else:
                    edge2.set_dir("forward")
                # Set the edge pen width
                if species_rate2 < species_rate_tol:
                    penwidth = 0.0
                    edge2.set_dir("none")
                else:
                    penwidth = round(slope * np.log10(species_rate2) + max_edge_pen_width, 3)
                edge2.set_penwidth(penwidth)
                graph.add_edge(edge2)

    # Go through edges in Mechanism 2
    for reactant_index2, product_index2 in edges2:
        # skip if this was already done in edges 1
        if [species2_to_1[reactant_index2], species2_to_1[product_index2]] in edges1 or \
            [species2_to_1[product_index2], species2_to_1[reactant_index2]] in edges1:
            continue

        if reactant_index2 in nodes2 and product_index2 in nodes2:
            # mech 2 says include this edge for all mechs
            if species2_to_1[reactant_index2] in nodes1:
                reactant2 = species_list1[species2_to_1[reactant_index2]]
            else:
                reactant2 = species_list2[reactant_index2]
            if species2_to_1[product_index2] in nodes1:
                product2 = species_list1[species2_to_1[product_index2]]
            else:
                product2 = species_list2[product_index2]
            edge2 = pydot.Edge(str(reactant2), str(product2), color=mech2_color)

            species_rate2 = species_rates2[t2, reactant_index2, product_index2] / max_species_rate_total
            if species_rate2 < 0:
                edge2.set_dir("back")
                species_rate2 = -species_rate2
            else:
                edge2.set_dir("forward")
            # Set the edge pen width
            if species_rate2 < species_rate_tol:
                penwidth = 0.0
                edge2.set_dir("none")
            else:
                penwidth = round(slope * np.log10(species_rate2) + max_edge_pen_width, 3)

            edge2.set_penwidth(penwidth)
            graph.add_edge(edge2)

            # add mech 1
            if species2_to_1[reactant_index2] >= 0 and species2_to_1[product_index2] >= 0:
                reactant_index1 = species2_to_1[reactant_index2]
                product_index1 = species2_to_1[product_index2]

                edge1 = pydot.Edge(str(reactant2), str(product2), color=mech1_color)
                species_rate1 = species_rates1[t1, reactant_index1, product_index1] / max_species_rate_total
                if species_rate1 < 0:
                    edge1.set_dir("back")
                    species_rate1 = -species_rate1
                else:
                    edge1.set_dir("forward")
                # Set the edge pen width
                if species_rate1 < species_rate_tol:
                    penwidth = 0.0
                    edge1.set_dir("none")
                else:
                    penwidth = round(slope * np.log10(species_rate1) + max_edge_pen_width, 3)
                edge1.set_penwidth(penwidth)
                graph.add_edge(edge1)


    # # General purpose graph settings
    # graph.set_nodesep(0.11)
    # graph.set_ranksep(0.35)
    # graph.set_rankdir('LR')

    # Add Legend
    graph.add_node(pydot.Node(mech_1_label + f'\nt={times1[t1]:.4e}', label=mech_1_label + f'\nt={times1[t1]:.4e}', color=mech1_color, shape='box', penwidth=max_node_pen_width))
    graph.add_node(pydot.Node(mech_2_label + f'\nt={times2[t2]:.4e}', label=mech_2_label + f'\nt={times2[t2]:.4e}', color=mech2_color, shape='box', penwidth=max_node_pen_width))


    # write in multiple formats
    graph.write_dot(os.path.join(diagram_base_name, f'{diagram_base_name}_{t1}.dot')) # Yes this is supposed to be an index, not an actual time
    graph.write_png(os.path.join(diagram_base_name, f'{diagram_base_name}_{t1}.png'))
    # graph.write_pdf(os.path.join(diagram_base_name, f'{diagram_base_name}_{t1}.pdf'))
    # graph.write_pdf(os.path.join(diagram_base_name, f'{diagram_base_name}_{t1}.svg'))